In [7]:
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types.{StructType, StructField, StringType, DoubleType};
import com.databricks.spark.xml._

val sqlContext = new SQLContext(sc)
import sqlContext.implicits._
sqlContext.sql("set spark.sql.caseSensitive=true") 
spark.sparkContext.uiWebUrl

sqlContext = org.apache.spark.sql.SQLContext@1a2d7d3a


lastException: Throwable = null


Some(http://iuni2:4041)

In [8]:
val WoS = spark.read.format("parquet").load("/WoSraw/WoS2019/parquet/*.parquet")

val WoS2 = WoS.select("UID","dynamic_data.cluster_related.identifiers.identifier","static_data.summary.doctypes.doctype",
                      "static_data.summary.names.name","static_data.summary.titles","static_data.summary.conferences.conference",
                      "static_data.fullrecord_metadata.addresses.address_name", "static_data.fullrecord_metadata.reprint_addresses",
                      "static_data.fullrecord_metadata.abstracts.abstract.abstract_text",
                      "static_data.fullrecord_metadata.category_info",
                      "static_data.fullrecord_metadata.keywords.keyword","static_data.fullrecord_metadata.references.reference",
                      "static_data.contributors.contributor","static_data.summary.EWUID",
                      "static_data.summary.pub_info","static_data.summary.publishers","static_data.fullrecord_metadata.fund_ack")
WoS2.printSchema

root
 |-- UID: string (nullable = true)
 |-- identifier: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |    |-- _value: string (nullable = true)
 |-- doctype: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _addr_no: string (nullable = true)
 |    |    |-- _orcid_id_tr: string (nullable = true)
 |    |    |-- _r_id_tr: string (nullable = true)
 |    |    |-- _reprint: string (nullable = true)
 |    |    |-- _role: string (nullable = true)
 |    |    |-- _seq_no: long (nullable = true)
 |    |    |-- display_name: string (nullable = true)
 |    |    |-- email_addr: string (nullable = true)
 |    |    |-- first_name: string (nullable = true)
 |    |    |-- full_name: string (nullable = true)
 |    |    |-- last_name: string (nullable = true)
 

WoS = [UID: string, _r_id_disclaimer: string ... 14 more fields]
WoS2 = [UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 15 more fields]


[UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 15 more fields]

In [9]:
WoS2.show(10, false)

+-------------------+-------------------------------------------------------------------------------------------------------------------+------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+-----------------------------------------------------------------------------------------------------

In [10]:
import org.apache.spark.sql.{Row, Column, SQLContext}
import org.apache.spark.sql.functions._
//WoS2.filter(WoS2("conference").isNull).count()
//WoS2.filter((WoS2($"conference") === "") || WoS2($"conference").isNull || WoS2($"conference").isNaN).count()
WoS2.select(WoS2.columns.map(colName => {
    count(when(col(colName).isNull, true)) as s"${colName}_nulls_count"
  }): _*)
.show(10)

+---------------+----------------------+-------------------+----------------+------------------+----------------------+------------------------+-----------------------------+-------------------------+-------------------------+-------------------+---------------------+-----------------------+-----------------+--------------------+----------------------+--------------------+
|UID_nulls_count|identifier_nulls_count|doctype_nulls_count|name_nulls_count|titles_nulls_count|conference_nulls_count|address_name_nulls_count|reprint_addresses_nulls_count|abstract_text_nulls_count|category_info_nulls_count|keyword_nulls_count|reference_nulls_count|contributor_nulls_count|EWUID_nulls_count|pub_info_nulls_count|publishers_nulls_count|fund_ack_nulls_count|
+---------------+----------------------+-------------------+----------------+------------------+----------------------+------------------------+-----------------------------+-------------------------+-------------------------+-------------------+--

In [11]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct
import org.apache.spark.sql.SaveMode

val WoSref = WoS2.select("UID","reference.uid").toDF("citing","refList").withColumn("cited", explode(col("refList")))
val WoScite = WoSref.select("citing","cited").orderBy($"cited".desc).cache()
WoScite.show

+-------------------+--------------------+
|             citing|               cited|
+-------------------+--------------------+
|WOS:000449272100019|ZOOREC:ZOOR156010...|
|WOS:000471106100011|ZOOREC:ZOOR156010...|
|WOS:000403194300008|ZOOREC:ZOOR156010...|
|WOS:000454765900002|ZOOREC:ZOOR156010...|
|WOS:000494949900001|ZOOREC:ZOOR156010...|
|WOS:000491655000001|ZOOREC:ZOOR156010...|
|WOS:000449763200002|ZOOREC:ZOOR156010...|
|WOS:000430169100002|ZOOREC:ZOOR156010...|
|WOS:000438645700005|ZOOREC:ZOOR156010...|
|WOS:000438021000006|ZOOREC:ZOOR156010...|
|WOS:000474431600008|ZOOREC:ZOOR156010...|
|WOS:000446382800002|ZOOREC:ZOOR156010...|
|WOS:000481758200009|ZOOREC:ZOOR156010...|
|WOS:000468859400021|ZOOREC:ZOOR156010...|
|WOS:000446382800003|ZOOREC:ZOOR156010...|
|WOS:000403818600008|ZOOREC:ZOOR156010...|
|WOS:000419185200017|ZOOREC:ZOOR156010...|
|WOS:000411286200005|ZOOREC:ZOOR156010...|
|WOS:000469254100001|ZOOREC:ZOOR156010...|
|WOS:000227281400004|ZOOREC:ZOOR156010...|
+----------

WoSref = [citing: string, refList: array<string> ... 1 more field]
WoScite = [citing: string, cited: string]


[citing: string, cited: string]

In [16]:
val sdf = spark.read.format("csv").option("header", "true").load("/data1/nobel_prize.csv")
//sdf.describe().filter($"summary" === "count").show
sdf.show

+---------+-----------+--------------+----------+--------------------+--------+----------+---------------+--------------------+--------------------+----------+---------+
|    field|laureate_id| laureate_name|prize_year|               title|pub_year|  paper_id|additional_info|                 doi|                 uid|nonenglish|no_record|
+---------+-----------+--------------+----------+--------------------+--------+----------+---------------+--------------------+--------------------+----------+---------+
| Medicine|      30174|     minot, gr|      1934|treatment of pern...|    1926|2032615709|           null|10.1001/jama.1926...| WOS:000201650200060|         0|        0|
| Medicine|      30174|     minot, gr|      1934|a diet rich in li...|    1927|2034899879|           null|10.1001/jama.1927...| WOS:000201650300139|         0|        0|
| Medicine|      30095|  blumberg, bs|      1976|a new antigen in ...|    1965|2082505865|           null|10.1001/jama.1965...| WOS:A19656141500006|  

sdf = [field: string, laureate_id: string ... 10 more fields]


[field: string, laureate_id: string ... 10 more fields]

In [18]:
sdf.describe().filter($"summary" === "count").show

+-------+-----+-----------+-------------+----------+-----+--------+--------+---------------+---+---+----------+---------+
|summary|field|laureate_id|laureate_name|prize_year|title|pub_year|paper_id|additional_info|doi|uid|nonenglish|no_record|
+-------+-----+-----------+-------------+----------+-----+--------+--------+---------------+---+---+----------+---------+
|  count|  731|        731|          731|       731|  731|     731|     730|              7|677|680|       731|      731|
+-------+-----+-----------+-------------+----------+-----+--------+--------+---------------+---+---+----------+---------+



In [52]:
val matchedPapers = sdf.join(WoS2, sdf("uid")===WoS2("UID"), "inner")
val matchedPapers2 = matchedPapers0.select("UID").distinct()
matchedPapers.describe().filter($"summary" === "count").show

+-------+-----+-----------+-------------+----------+-----+--------+--------+---------------+---+---+----------+---------+---+
|summary|field|laureate_id|laureate_name|prize_year|title|pub_year|paper_id|additional_info|doi|uid|nonenglish|no_record|UID|
+-------+-----+-----------+-------------+----------+-----+--------+--------+---------------+---+---+----------+---------+---+
|  count|  674|        674|          674|       674|  674|     674|     674|              4|626|674|       674|      674|674|
+-------+-----+-----------+-------------+----------+-----+--------+--------+---------------+---+---+----------+---------+---+



matchedPapers = [field: string, laureate_id: string ... 27 more fields]
matchedPapers2 = [UID: string]


lastException: Throwable = null


[UID: string]

In [58]:
import org.apache.spark.sql.functions.not
import org.apache.spark.sql.functions.lit

val matchedRefs = sdf.join(WoScite, sdf("uid")===WoScite("citing"), "inner").select("cited")
                .where($"cited".contains("WOS:") && not($"cited".contains("."))).distinct()
val matchedRefs2 = matchedRefs.withColumn("set", lit("references"))
//matchedRefs2.show(false)
matchedRefs2.describe().filter($"summary" === "count").show

+-------+-----+----+
|summary|cited| set|
+-------+-----+----+
|  count| 9004|9004|
+-------+-----+----+



matchedRefs = [cited: string]
matchedRefs2 = [cited: string, set: string]


[cited: string, set: string]

In [59]:
val matchedCites = sdf.join(WoScite, sdf("uid")===WoScite("cited"), "inner").select("citing").distinct()
val matchedCites2 = matchedCites.withColumn("set", lit("citingPapers"))
//matchedCites2.show(false)
matchedCites2.describe().filter($"summary" === "count").show

+-------+------+------+
|summary|citing|   set|
+-------+------+------+
|  count|898073|898073|
+-------+------+------+



matchedCites = [citing: string]
matchedCites2 = [citing: string, set: string]


[citing: string, set: string]

In [93]:
val matchedRefs = sdf.join(WoScite, sdf("uid")===WoScite("citing"), "inner").select("citing","cited")
val matchedCites = sdf.join(WoScite, sdf("uid")===WoScite("cited"), "inner").select("citing","cited")
val edgeList = matchedRefs.union(matchedCites).cache()
edgeList.describe().filter($"summary" === "count").show

+-------+-------+-------+
|summary| citing|  cited|
+-------+-------+-------+
|  count|1121152|1121044|
+-------+-------+-------+



matchedRefs = [citing: string, cited: string]
matchedCites = [citing: string, cited: string]
edgeList = [citing: string, cited: string]


lastException: Throwable = null


[citing: string, cited: string]

In [94]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct
import org.apache.spark.sql.SaveMode

val session = SparkSession.builder().getOrCreate()

val azureFile = sc.textFile("/azureBlob.txt")
session.conf.set(
    "fs.azure.account.key.iunimag.blob.core.windows.net",
     azureFile.collect().mkString
)

edgeList.coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("quote", "\"")
    .save("wasbs://mag-2020-07-29@iunimag.blob.core.windows.net/2020-08-03/nobel_edges.csv")

session = org.apache.spark.sql.SparkSession@fc0ce75
azureFile = /azureBlob.txt MapPartitionsRDD[861] at textFile at <console>:91


/azureBlob.txt MapPartitionsRDD[861] at textFile at <console>:91

In [57]:
val paperList = matchedPapers2.withColumn("set", lit("focalPapers"))
val paperList2 = paperList.union(matchedCites2).union(matchedRefs2)
paperList2.describe().filter($"summary" === "count").show

+-------+------+------+
|summary|   UID|   set|
+-------+------+------+
|  count|907745|907745|
+-------+------+------+



paperList = [UID: string, set: string]
paperList2 = [UID: string, set: string]


[UID: string, set: string]

In [72]:
val WoS2b = paperList2.join(WoS2, Seq("UID"), "inner")
val WoS3 = WoS2b.select("set", "UID", "pub_info._pubtype", "pub_info._pubyear", "fund_ack.fund_text", "titles", "conference", "keyword", "abstract_text",
                       "identifier", "name","publishers.publisher.names.name.full_name","address_name.address_spec.full_address","category_info").cache()
WoS3.describe().filter($"summary" === "count").show

+-------+------+------+--------+--------+---------+
|summary|   set|   UID|_pubtype|_pubyear|full_name|
+-------+------+------+--------+--------+---------+
|  count|907752|907752|  907752|  907752|   903741|
+-------+------+------+--------+--------+---------+



WoS2b = [UID: string, set: string ... 16 more fields]
WoS3 = [set: string, UID: string ... 12 more fields]


lastException: Throwable = null


[set: string, UID: string ... 12 more fields]

In [75]:
val WoS4 = WoS3.withColumn("FX", array_join($"fund_text.p", ".")).withColumn("AB", array_join($"abstract_text.p", "."))
WoS4.describe().filter($"summary" === "count").show

+-------+------+------+--------+--------+---------+------+------+
|summary|   set|   UID|_pubtype|_pubyear|full_name|    FX|    AB|
+-------+------+------+--------+--------+---------+------+------+
|  count|907752|907752|  907752|  907752|   903741|213552|583287|
+-------+------+------+--------+--------+---------+------+------+



WoS4 = [set: string, UID: string ... 14 more fields]


[set: string, UID: string ... 14 more fields]

In [76]:
val WoS5 = WoS4.withColumn("allAddresses", array_join($"full_address", "|"))
//_addr_no map to author_no required for author-address mapping
WoS5.show

+------------+-------------------+--------+--------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+
|         set|                UID|_pubtype|_pubyear|fund_text|              titles|          conference|             keyword|       abstract_text|          identifier|                name|           full_name|        full_address|       category_info|  FX|                  AB|        allAddresses|
+------------+-------------------+--------+--------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+
|citingPapers|WOS:000071146300041| Journal|    1998|     null|[6, [[ENDOCRINOLO...|                null

WoS5 = [set: string, UID: string ... 15 more fields]


[set: string, UID: string ... 15 more fields]

In [77]:
import scala.util.Try
import org.apache.hadoop.fs._
import org.apache.spark.sql.Row

def getTitle(_type: String) = { 
  udf((row: Seq[Row]) =>
    row.map(r => (r.getString(0), r.getString(1), r.getString(2)))
      .find { case (_, _, s) => s == _type}
      .map(_._1)
      .orElse(None)
  )
}
val WoS6 = WoS5.withColumn("journal", when(col("titles.title").isNotNull, getTitle("source")($"titles.title")).otherwise(lit(null))).cache()
WoS6.show()

+------------+-------------------+--------+--------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+
|         set|                UID|_pubtype|_pubyear|fund_text|              titles|          conference|             keyword|       abstract_text|          identifier|                name|           full_name|        full_address|       category_info|  FX|                  AB|        allAddresses|             journal|
+------------+-------------------+--------+--------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+
|citingPapers|WOS:000071146300041| Journ

WoS6 = [set: string, UID: string ... 16 more fields]


getTitle: (_type: String)org.apache.spark.sql.expressions.UserDefinedFunction


[set: string, UID: string ... 16 more fields]

In [78]:
val WoS7 = WoS6.withColumn("title", when(col("titles.title").isNotNull, getTitle("item")($"titles.title")).otherwise(lit(null)))
WoS7.show

+------------+-------------------+--------+--------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------+--------------------+
|         set|                UID|_pubtype|_pubyear|fund_text|              titles|          conference|             keyword|       abstract_text|          identifier|                name|           full_name|        full_address|       category_info|  FX|                  AB|        allAddresses|             journal|               title|
+------------+-------------------+--------+--------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+--------------------

WoS7 = [set: string, UID: string ... 17 more fields]


[set: string, UID: string ... 17 more fields]

In [79]:
def getID(_type: String) = {
  udf((row: Seq[Row]) =>
    row.map(r => (r.getString(0), r.getString(1), r.getString(2)))
      .find { case (_, s, _) => s == _type}
      .map(_._3)
      .orElse(None)
  )
}


val WoS8 = WoS7.withColumn("doi", when(col("identifier").isNotNull, getID("doi")($"identifier")).otherwise(lit(null)))
.withColumn("issn", when(col("identifier").isNotNull, getID("issn")($"identifier")).otherwise(lit(null)))
.withColumn("eissn", when(col("identifier").isNotNull, getID("eissn")($"identifier")).otherwise(lit(null))).cache()
//.select("UID","doi","issn","eissn").filter($"doi" =!= "null").show
WoS8.describe().filter($"summary" === "count").show

+-------+------+------+--------+--------+---------+------+------+------------+-------+------+------+------+------+
|summary|   set|   UID|_pubtype|_pubyear|full_name|    FX|    AB|allAddresses|journal| title|   doi|  issn| eissn|
+-------+------+------+--------+--------+---------+------+------+------------+-------+------+------+------+------+
|  count|907752|907752|  907752|  907752|   903741|213552|583287|      711273| 907752|907751|358506|874964|325329|
+-------+------+------+--------+--------+---------+------+------+------------+-------+------+------+------+------+



WoS8 = [set: string, UID: string ... 20 more fields]


getID: (_type: String)org.apache.spark.sql.expressions.UserDefinedFunction


[set: string, UID: string ... 20 more fields]

In [84]:
val WoS9 = WoS8.withColumn("reprintFlag", array_position($"name._reprint", "Y"))
            .select("set","UID","_pubtype","_pubyear","eissn","issn","doi","title","AB","journal","full_name","FX","reprintFlag",
                    "name.display_name","allAddresses","conference.conf_titles.conf_title","keyword",
                    "category_info.headings.heading","category_info.subheadings.subheading","category_info.subjects.subject")
            .toDF("set","WoSid","PT","PY","EI","SN","DI","TI","AB","SO","PU","FX","RP","authors","C1","conference","keyword","heading","subheading","subject")
            .withColumn("AU", array_join($"authors", "|"))
WoS9.show(false)

+------------+-------------------+-------+----+---------+---------+----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

WoS9 = [set: string, WoSid: string ... 19 more fields]


lastException: Throwable = null


[set: string, WoSid: string ... 19 more fields]

In [85]:
val WoS10 = WoS9.withColumn("DE", array_join($"keyword", ".")).withColumn("CT", array_join($"conference", "."))
//WoS9.printSchema
WoS10.describe().filter($"summary" === "count").show

+-------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+
|summary|   set| WoSid|    PT|    PY|    EI|    SN|    DI|    TI|    AB|    SO|    PU|    FX|    RP|    C1|    AU|    DE|   CT|
+-------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+
|  count|907752|907752|907752|907752|325329|874964|358506|907751|583287|907752|903741|213552|907752|711273|907752|293397|48090|
+-------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+



WoS10 = [set: string, WoSid: string ... 21 more fields]


[set: string, WoSid: string ... 21 more fields]

In [86]:
def getSubject(_type: String) = { 
  udf((row: Seq[Row]) =>
    row.map(r => (r.getString(0), r.getString(1)))
      .find { case (_, s) => s == _type}
      .map(_._1)
      .orElse(None)
  )
}

val WoS11 = WoS10.withColumn("headings", array_join($"heading", ","))
                .withColumn("subheadings", array_join($"subheading", ","))
                //.withColumn("WC/SC", array_join($"subject", ";"))
                .withColumn("WC", when(col("subject").isNotNull, getSubject("traditional")($"subject")).otherwise(lit(null)))
                .withColumn("SC", when(col("subject").isNotNull, getSubject("extended")($"subject")).otherwise(lit(null)))
WoS11.show(false)

+------------+-------------------+-------+----+---------+---------+----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

WoS11 = [set: string, WoSid: string ... 25 more fields]


getSubject: (_type: String)org.apache.spark.sql.expressions.UserDefinedFunction


[set: string, WoSid: string ... 25 more fields]

In [87]:
WoS11.printSchema

root
 |-- set: string (nullable = false)
 |-- WoSid: string (nullable = true)
 |-- PT: string (nullable = true)
 |-- PY: long (nullable = true)
 |-- EI: string (nullable = true)
 |-- SN: string (nullable = true)
 |-- DI: string (nullable = true)
 |-- TI: string (nullable = true)
 |-- AB: string (nullable = true)
 |-- SO: string (nullable = true)
 |-- PU: string (nullable = true)
 |-- FX: string (nullable = true)
 |-- RP: long (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- C1: string (nullable = true)
 |-- conference: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- keyword: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- heading: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- subheading: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- subject: array (nullable = true)
 |    |-- element: struct (containsNull = tr

In [88]:
WoS11.select("set","WoSid","PT","PY","EI","SN","DI","TI","AB","DE","CT","SO","PU","FX","RP","AU","C1"
             ,"headings","subheadings","WC","SC")
    .coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true").option("compression", "gzip")
    .save("/data1/nobel_papers.csv")

In [89]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct
import org.apache.spark.sql.SaveMode

val session = SparkSession.builder().getOrCreate()

val azureFile = sc.textFile("/azureBlob.txt")
session.conf.set(
    "fs.azure.account.key.iunimag.blob.core.windows.net",
     azureFile.collect().mkString
)

WoS11.select("set","WoSid","PT","PY","EI","SN","DI","TI","AB","DE","CT","SO","PU","FX","RP","AU","C1"
             ,"headings","subheadings","WC","SC")
    .coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("quote", "\"")
    .save("wasbs://mag-2020-07-29@iunimag.blob.core.windows.net/2020-08-03/nobel_papers.csv")

session = org.apache.spark.sql.SparkSession@fc0ce75
azureFile = /azureBlob.txt MapPartitionsRDD[786] at textFile at <console>:81


/azureBlob.txt MapPartitionsRDD[786] at textFile at <console>:81